Let's do all preprocessing required for fitting the seq-2-seq model here. We need to extract features for the subwindows. We also need to write generic code incase we need to do changes later depending on the performance of the seq-2-seq model.

For the seq-2-seq model, we do not have to touch the densities, and as the first step we should extract the features from each subwindow using the fine-tuned model. We still might need the densities later for the latent AR proces part, we will get there in stage 2. We will need to arange the subwindows in a stack though, so that the features could be extracted. We ahve done this sort of work in AR models, we will look into that too.

In [1]:
import numpy as np
import tensorflow as tf
import os
from skimage.transform import resize
import matplotlib.pyplot as plt

2025-06-03 20:24:48.428069: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-03 20:24:48.726233: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-03 20:24:48.726287: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-03 20:24:48.793301: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-03 20:24:48.895220: I tensorflow/core/platform/cpu_feature_guar

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

##### Train and Validation data

Let's first focus on train and validation data. Meaning blocks 101, 102, 203, 301, and 204. We will preprocess data required for seq-2-seq model, we do not have to worry currently about the stage 2 (densities). But how are we going to evaluate the seq-2-seq model? should we do that on the rest of the blocks? Therefore we might have to preprocess the test data too. Let's get block 0101 as the example, develop all functions for this, and work on preprocessing data

For train data, we need to prepare our final input data in the shape None, 13, 32, and the target data by None, 7, 32. For this, we need to use the fin-tuned model to extract the data. 

We will also keep pasting the generic functions here, the ones we can use to preprocess the data easily for all the blocks below.

In [3]:
# Write a function for extracting onlt the horizontal images out of all the files in the location

def get_horizontal_images(file_path):
    # get all contents at the file path
    all_files = os.listdir(file_path)
    # get only the image files as we do not need the xml files for the seq2seq model
    only_images = [file for file in all_files if file.split(".")[-1] == 'jpeg']
    # from all the images only choose the horizontal ones
    horizontal_images = [file for file in only_images if file.split('.')[0][-10:] in horizontal_image_list]
    # sort the list
    horizontal_images.sort()
    # return the image list
    return horizontal_images

In [4]:
def create_sub_windows_and_extract_features(folder_path, file, stride = 32, kernel_size = 32):
    # joined image path
    joined_im_path = os.path.join(folder_path, file)
    # read the image
    loaded_im_file = plt.imread(joined_im_path)
    # create subwindows and get prediction
    img_height = loaded_im_file.shape[0]
    img_width = loaded_im_file.shape[1]

    # catch alle extracted features here
    all_extracted_features = []
    # you can also keep track the subwindows here if required - but let's not worry about that for now
    for i in  range(0, img_height, stride):
        for j in range(0, img_width, stride):
            sub_window = loaded_im_file[i: i + kernel_size, j : j + kernel_size,:]
            # resize the subwindow - for 300*300
            sub_window = resize(sub_window, (kernel_size, kernel_size,3))
            # expand the dimension of the window to get predictions from the feature extractor - (make subwindows from shape (32,32,3) to
            # (1,32,32,3))
            sub_window = np.expand_dims(sub_window, axis = 0)
            # extract the features
            predicted_features = feature_extractor_model.predict(sub_window)
            # append the extracted features
            all_extracted_features.append(predicted_features)

    return all_extracted_features  

##### Block 0101

In [5]:
# location of the images and the correct xml files

blk_0101_data_path = '../../Spring_2024/S_lab_TasselNet/Block_1_TN/Block_1_images_and_xml'

In [6]:
content_here = os.listdir(blk_0101_data_path)
only_images = [file for file in content_here if file.split(".")[-1] == 'jpeg']

In [7]:
# len(only_images)

In [8]:
# Okay, at the end how do we need the data look like? - None, 13, 32

# What do we use for this? We use the images we have - let's only pich the horizontal images out of the lot of files we have at the location.

In [9]:
horizontal_image_list = ['2020_08_03', '2020_08_04', '2020_08_06', '2020_08_07', '2020_08_11', '2020_08_12', '2020_08_14', '2020_08_15', '2020_08_17', '2020_08_18', '2020_08_19', '2020_08_21', '2020_08_25', '2020_08_26', '2020_08_27', '2020_08_28', '2020_08_31', '2020_09_02', '2020_09_07', '2020_09_16']

In [10]:
# len(horizontal_image_list)

In [11]:
only_images[0].split('.')[0][-10:]

'2020_08_15'

In [12]:
blk_0101_horizontal_images = [file for file in only_images if file.split('.')[0][-10:] in horizontal_image_list]
# sort these files according to date
blk_0101_horizontal_images.sort()

In [13]:
# blk_0101_horizontal_images
len(blk_0101_horizontal_images)

20

In [14]:
# Write a function for extracting onlt the horizontal images out of all the files in the location

def get_horizontal_images(file_path):
    # get all contents at the file path
    all_files = os.listdir(file_path)
    # get only the image files as we do not need the xml files for the seq2seq model
    only_images = [file for file in all_files if file.split(".")[-1] == 'jpeg']
    # from all the images only choose the horizontal ones
    horizontal_images = [file for file in only_images if file.split('.')[0][-10:] in horizontal_image_list]
    # sort the list
    horizontal_images.sort()
    # return the image list
    return horizontal_images
    

In [15]:
# try out the function
blk_0101_horizontal_image_files = get_horizontal_images(blk_0101_data_path)

In [16]:
# blk_0101_horizontal_image_files
len(blk_0101_horizontal_image_files)

20

In [17]:
# function seem to work
# sanity check
np.mean(blk_0101_horizontal_images == blk_0101_horizontal_image_files)

1.0

In [18]:
# Okay, what now? We can write a function for extracting features below - we might want to think how exactly we need to stack the contents
# but notice that we do not want to store the subwindow (32,32,3) at all, and can just use the model.predict function to extract the features

In [19]:
# load the finetuned gmp model
gmp_model = tf.keras.models.load_model('../../Spring_2024/Bayes_for_comps/TS_bayes_implementation_for_TN/models/trained_gmp_model_dense_32_new.h5')

2025-06-03 20:25:00.607542: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


In [20]:
gmp_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                           

In [21]:
# define the feature extractor

# feature extractor input
feat_ext_input = gmp_model.input

# feature extractor output - do this at the ReLu activation layer - as this will give the same features as the dropout layer (It does not matter if it is the dropout or the activation layer, the extracted features will be the same)
feat_ext_output = gmp_model.layers[-4].output

feature_extractor_model = tf.keras.models.Model(inputs = feat_ext_input, outputs = feat_ext_output)

In [22]:
feature_extractor_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                             

In [23]:
# Let us read one horizontal image
joined_im_path = os.path.join(blk_0101_data_path, blk_0101_horizontal_image_files[0])
# load the image and the count numpy files
loaded_im_file = plt.imread(joined_im_path)

In [24]:
loaded_im_file.shape

(768, 1024, 3)

In [25]:
sub_window = loaded_im_file[0: 0 + 32, 0 : 0 + 32,:]
sub_window = resize(sub_window, (32, 32,3))

In [26]:
sub_window = np.expand_dims(sub_window, axis = 0)

In [27]:
sub_window.shape

(1, 32, 32, 3)

In [28]:
# try prediction from the feature extractor
try_predict = feature_extractor_model.predict(sub_window)

2025-06-03 20:25:01.522870: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


1/1 [==============================] - 2s 2s/step


In [29]:
try_predict.shape

(1, 32)

In [30]:
try_predict

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.1349885 , 0.        , 0.12475819, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.16891207, 0.        ,
        0.        , 0.        , 0.18322447, 0.        , 0.16411296,
        0.1503782 , 0.        , 0.        , 0.        , 0.1410548 ,
        0.        , 0.14434241, 0.        , 0.        , 0.1251781 ,
        0.12773699, 0.        ]], dtype=float32)

In [31]:
sub_window_alt = loaded_im_file[0: 0 + 32, 0 : 0 + 32,:]
sub_window_alt = resize(sub_window_alt, (32, 32,3))
sub_window_alt = sub_window_alt[None,  ...]

In [32]:
sub_window_alt.shape

(1, 32, 32, 3)

In [33]:
np.mean(sub_window == sub_window_alt)

1.0

In [34]:
# we have the horizontal image list here - blk_0101_horizontal_image_files

In [35]:
# blk_0101_horizontal_image_files

In [36]:
# Once we have our data, we will just do the toy model we have to train the seq2seq model, we will wokr on some better models after the meeting tomorrom (06/04/2025)

In [37]:
def create_sub_windows_and_extract_features(folder_path, file, stride = 32, kernel_size = 32):
    # joined image path
    joined_im_path = os.path.join(folder_path, file)
    # read the image
    loaded_im_file = plt.imread(joined_im_path)
    # create subwindows and get prediction
    img_height = loaded_im_file.shape[0]
    img_width = loaded_im_file.shape[1]

    # catch alle extracted features here
    all_extracted_features = []
    # you can also keep track the subwindows here if required - but let's not worry about that for now
    for i in  range(0, img_height, stride):
        for j in range(0, img_width, stride):
            sub_window = loaded_im_file[i: i + kernel_size, j : j + kernel_size,:]
            # resize the subwindow - for 300*300
            sub_window = resize(sub_window, (kernel_size, kernel_size,3))
            # expand the dimension of the window to get predictions from the feature extractor - (make subwindows from shape (32,32,3) to
            # (1,32,32,3))
            sub_window = np.expand_dims(sub_window, axis = 0)
            # extract the features
            predicted_features = feature_extractor_model.predict(sub_window)
            # append the extracted features
            all_extracted_features.append(predicted_features)

    return all_extracted_features  

In [38]:
# Let's try this for a single image?

In [39]:
%%time
# let's go with subwindows of size - 30*30
try_extracting_features_im_0 = create_sub_windows_and_extract_features(blk_0101_data_path, blk_0101_horizontal_image_files[0], 30, 30)

1/1 [==============================] - 0s 15ms/step
CPU times: user 44.4 s, sys: 2.85 s, total: 47.2 s
Wall time: 43.1 s


In [40]:
len(try_extracting_features_im_0)

910

In [41]:
try_extracting_features_im_0[0].shape

(1, 32)

In [42]:
# we might want to stack the contents here 
try_stack = np.stack(try_extracting_features_im_0, axis = 0)

In [43]:
try_stack.shape

(910, 1, 32)

In [44]:
im_00 = try_stack[0, :, :]

In [45]:
im_00.shape

(1, 32)

In [46]:
%%time
# Maybe do this in a loop
append_means = []
for i in range(len(try_extracting_features_im_0)):
    subwindow_features = try_extracting_features_im_0[i]
    append_means.append(np.mean(subwindow_features == try_stack[i, :, :]))
    print(np.mean(subwindow_features == try_stack[i, :, :]))

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [47]:
# if this is done right, we will get 1 as the mean of the list
np.mean(append_means)

1.0

In [48]:
# We definitely need some sanity checks for this to verify whatever we are doing is the correct thing - but for now this seems okay.

In [49]:
# Sanity check complete, therefore we are doing the stacking correct

In [50]:
# Should we trying doing this for the entire block? - but maybe separately as the input and output time points - that way we do not have to separate matrices later 

In [51]:
blk_0101_horizontal_image_files

['Block0101_2020_08_03.jpeg',
 'Block0101_2020_08_04.jpeg',
 'Block0101_2020_08_06.jpeg',
 'Block0101_2020_08_07.jpeg',
 'Block0101_2020_08_11.jpeg',
 'Block0101_2020_08_12.jpeg',
 'Block0101_2020_08_14.jpeg',
 'Block0101_2020_08_15.jpeg',
 'Block0101_2020_08_17.jpeg',
 'Block0101_2020_08_18.jpeg',
 'Block0101_2020_08_19.jpeg',
 'Block0101_2020_08_21.jpeg',
 'Block0101_2020_08_25.jpeg',
 'Block0101_2020_08_26.jpeg',
 'Block0101_2020_08_27.jpeg',
 'Block0101_2020_08_28.jpeg',
 'Block0101_2020_08_31.jpeg',
 'Block0101_2020_09_02.jpeg',
 'Block0101_2020_09_07.jpeg',
 'Block0101_2020_09_16.jpeg']

In [52]:
# Since the horizontal images are in the order of the time points, splitting is straight-forward.
blk_0101_input_time_point_images = blk_0101_horizontal_image_files[:13]
blk_0101_input_time_point_images.sort()
blk_0101_output_time_point_images = blk_0101_horizontal_image_files[-7:]
blk_0101_output_time_point_images.sort()

In [53]:
# blk_0101_input_time_point_images, blk_0101_output_time_point_images

In [ ]:
%%time
# extract features for the input time points
blk_0101_input_extracted_features = []
counter = 1
for file in blk_0101_input_time_point_images:
    blk_0101_extracted_features = create_sub_windows_and_extract_features(blk_0101_data_path, file, stride = 30, kernel_size = 30)
    # stack these?
    blk_0101_feature_stack = np.stack(blk_0101_extracted_features, axis = 0)
    # append these
    blk_0101_input_extracted_features.append(blk_0101_feature_stack)
    print("Extracted features for image: ", counter)
    counter = counter + 1
    

1/1 [==============================] - 0s 15ms/step
Extracted features for image:  1
1/1 [==============================] - 0s 16ms/step
Extracted features for image:  2
1/1 [==============================] - 0s 16ms/step
Extracted features for image:  3
1/1 [==============================] - 0s 23ms/step
Extracted features for image:  4
1/1 [==============================] - 0s 15ms/step
Extracted features for image:  5
1/1 [==============================] - 0s 16ms/step
Extracted features for image:  6
1/1 [==============================] - 0s 15ms/step
Extracted features for image:  7
1/1 [==============================] - 0s 15ms/step
Extracted features for image:  8
1/1 [==============================] - 0s 16ms/step
Extracted features for image:  9
1/1 [==============================] - 0s 16ms/step
Extracted features for image:  10
1/1 [==============================] - 0s 15ms/step
Extracted features for image:  11
1/1 [==============================] - 0s 23ms/step


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



1/1 [==============================] - 0s 15ms/step


In [63]:
len(blk_0101_input_extracted_features)

13

In [64]:
# now how to stack these?
# I think we stack across axis = 1 in the np.stack

blk_0101_input_complete_stack = np.hstack(blk_0101_input_extracted_features)

# check the shape just to be sure?
blk_0101_input_complete_stack.shape

(910, 13, 32)

In [65]:
# We can still do some random sanity checks to make sure we are stacking and book-keeping things right

In [66]:
%%time
# chose the random images 3, 8, and 11
try_extracting_features_im_3 = create_sub_windows_and_extract_features(blk_0101_data_path, blk_0101_horizontal_image_files[2], 30, 30)
try_extracting_features_im_8 = create_sub_windows_and_extract_features(blk_0101_data_path, blk_0101_horizontal_image_files[7], 30, 30)
try_extracting_features_im_11 = create_sub_windows_and_extract_features(blk_0101_data_path, blk_0101_horizontal_image_files[10], 30, 30)
# make sure we have this also for last one
try_extracting_features_im_13 = create_sub_windows_and_extract_features(blk_0101_data_path, blk_0101_horizontal_image_files[12], 30, 30)

1/1 [==============================] - 0s 16ms/step
CPU times: user 3min 4s, sys: 11.6 s, total: 3min 16s
Wall time: 2min 59s


In [67]:
print(np.mean(blk_0101_input_complete_stack[:, 2:3, :] == try_extracting_features_im_3))
print(np.mean(blk_0101_input_complete_stack[:, 7:8, :] == try_extracting_features_im_8))
print(np.mean(blk_0101_input_complete_stack[:, 10:11, :] == try_extracting_features_im_11))
print(np.mean(blk_0101_input_complete_stack[:, 12:13, :] == try_extracting_features_im_13))

1.0
1.0
1.0
1.0


In [61]:
len(blk_0101_input_extracted_features)

13

In [68]:
blk_0101_input_complete_stack.shape

(910, 13, 32)

In [75]:
blk_0101_input_complete_stack[:, 12:13, :][550]

array([[0.       , 0.       , 0.       , 0.       , 0.       , 1.6112341,
        0.       , 1.468425 , 0.       , 0.       , 0.7262168, 0.       ,
        0.       , 1.9115223, 0.       , 0.       , 0.       , 1.9811039,
        0.       , 1.8859277, 1.7077601, 0.       , 0.       , 0.       ,
        1.6459869, 0.       , 1.7183166, 0.       , 0.       , 1.6627644,
        1.593149 , 0.       ]], dtype=float32)

In [ ]:
# # # Since this is taking a lot of time, parallelize this task before moving forward?
# import concurrent.futures
# import numpy as np

# # Example operation that takes two arguments: matrix and a multiplier
# def operation_on_matrix(matrix, multiplier):
#     return matrix * multiplier

# # List of matrices (replace with your actual matrices)
# matrices = [np.random.rand(3, 3) for _ in range(10)]

# # Multiplier for the operation (example)
# multiplier = 2

# # Using ThreadPoolExecutor to parallelize the loop with multiple arguments
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     # Pass matrices and multiplier to the operation function
#     results = list(executor.map(lambda matrix: operation_on_matrix(matrix, multiplier), matrices))

# # Stack the results into a single 3D array (preserving the order)
# output_matrix = np.stack(results, axis=0)

# # Check the shape of the stacked matrix
# print(output_matrix.shape) 

In [81]:
import concurrent.futures

In [82]:
%%time
# Using ThreadPoolExecutor to parallelize the loop with multiple arguments
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Pass matrices and multiplier to the operation function
    results = list(executor.map(lambda file: create_sub_windows_and_extract_features(blk_0101_data_path, file, stride = 30, kernel_size = 30), blk_0101_input_time_point_images))

1/1 [==============================] - 0s 17ms/step
CPU times: user 21min 54s, sys: 1min 28s, total: 23min 23s
Wall time: 11min 31s


In [83]:
# stack the reults

In [84]:
parallelized_output = np.hstack(results)

# check the shape just to be sure?
parallelized_output.shape

(910, 13, 32)

In [85]:
np.mean(blk_0101_input_complete_stack == parallelized_output)

1.0

In [88]:
# Save the extracted features?
np.save("seq_2_seq_train_data/block_0101_extracted_input_features.npy", blk_0101_input_complete_stack)

In [89]:
# Do the same for the target time points

In [90]:
%%time
# extract features for the target time points
blk_0101_target_features = []
counter = 1
for file in blk_0101_output_time_point_images:
    blk_0101_extracted_features = create_sub_windows_and_extract_features(blk_0101_data_path, file, stride = 30, kernel_size = 30)
    # stack these?
    blk_0101_feature_stack = np.stack(blk_0101_extracted_features, axis = 0)
    # append these
    blk_0101_target_features.append(blk_0101_feature_stack)
    counter = counter + 1
    print("Extracting features for image: ", counter)

1/1 [==============================] - 0s 15ms/step
Extracting features for image:  2
1/1 [==============================] - 0s 15ms/step
Extracting features for image:  3
1/1 [==============================] - 0s 15ms/step
Extracting features for image:  4
1/1 [==============================] - 0s 15ms/step
Extracting features for image:  5
1/1 [==============================] - 0s 16ms/step
Extracting features for image:  6
1/1 [==============================] - 0s 15ms/step
Extracting features for image:  7
1/1 [==============================] - 0s 16ms/step
Extracting features for image:  8
CPU times: user 5min 23s, sys: 21 s, total: 5min 44s
Wall time: 5min 13s


In [91]:
%%time
# chose the random images 1, 4, and 7
try_extracting_features_im_1 = create_sub_windows_and_extract_features(blk_0101_data_path, blk_0101_horizontal_image_files[13], 30, 30)
try_extracting_features_im_4 = create_sub_windows_and_extract_features(blk_0101_data_path, blk_0101_horizontal_image_files[16], 30, 30)
try_extracting_features_im_7 = create_sub_windows_and_extract_features(blk_0101_data_path, blk_0101_horizontal_image_files[19], 30, 30)

1/1 [==============================] - 0s 16ms/step
CPU times: user 2min 18s, sys: 8.89 s, total: 2min 27s
Wall time: 2min 14s


In [92]:
blk_0101_target_complete_stack = np.hstack(blk_0101_target_features)

# check the shape just to be sure?
blk_0101_target_complete_stack.shape

(910, 7, 32)

In [93]:
print(np.mean(blk_0101_target_complete_stack[:, 0:1, :] == try_extracting_features_im_1))
print(np.mean(blk_0101_target_complete_stack[:, 3:4, :] == try_extracting_features_im_4))
print(np.mean(blk_0101_target_complete_stack[:, 6:7, :] == try_extracting_features_im_7))

1.0
1.0
1.0


In [94]:
# Save the extracted features?
np.save("seq_2_seq_train_data/block_0101_extracted_target_features.npy", blk_0101_target_complete_stack)

#### No need for tomorrow, but definitely consider doing a batch prediction using the data generator, for this, we might need the daya in batches - so like maybe all subwindows stacked together before passing it to model.predict - might have to redo the functions. But this might save time, try this for the test data?